In [ ]:
import numpy as np
import seaborn as sb
import pandas
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
%matplotlib notebook

# Linear regression - one variable

Let's do a simple linear regression on the boston housing data. This includes 13 descriptors for different houses that can be used to try to predict the observed housing price.

Let's first load it and take a look at the correlation matrix:

In [ ]:
boston = load_boston()
print(boston.DESCR)
# get correlation matrix:
c = np.corrcoef(boston.data.T)
# use seaborn to visualize the matrix:
fig, ax = plt.subplots(figsize=(8,8)) 
sb.heatmap(c, vmin=-1., vmax=1., square=True, annot=True, fmt='.2f', ax=ax).xaxis.tick_top()

There are a LOT of descriptor correlations here. In addition, we see that the fourth descriptor is very only weakly correlated with anything. Let's take a closer look:

In [ ]:
sb.pairplot(pandas.DataFrame(boston.data[:,0:5],columns=boston.feature_names[0:5]),size=1.5)

Ouch. Descriptor "CHAS" is actually a categorical variable! In addition, we see that other factors (such as "CRIM") have very skewed distributions. This is certainly a challenging dataset! 

So, let's take one of the features and let's try to predict `boston.target`, i.e., the housing prices from it with a linear model:

In [ ]:
# one variable from the housing data - since we only have one feature, we need to
# reshape the data, so that LinearRegression properly fits!
lstat = boston.data[:,-1].reshape(-1,1)
# do the fitting 
lr = LinearRegression().fit(lstat,boston.target)
# plot the results
fig, ax = plt.subplots(figsize=(4,4))
plt.scatter(lstat,boston.target)
# use predict to conveniently plot the line!
plt.plot(lstat,lr.predict(lstat),'k')
plt.xlabel('lstat')
plt.ylabel('housing price')
# equation of the line and explained r^2 in title
plt.title('hp = {:.3f} * ls + {:.3f} | R^2={:.3f}'.format(lr.coef_[0],lr.intercept_,lr.score(lstat,boston.target)))


So, we can see that there seems to be a good predictability from "LSTAT", that is, the number of lower status households in a neighborhood. We can also see that

* there seems to be a cap to housing prices at $50K, providing a potentially problematic ceiling
* the relationship between "LSTAT" and housing price may actually be non-linear!


To address the second point, let's do something very simple. Let's transform our "LSTAT" variable logarithmically. A logarithmic transform is useful if the distribution of the variable is skewed!

In [ ]:
# one variable from the housing data - since we only have one feature, we need to
# reshape the data, so that LinearRegression properly fits!
lstat = boston.data[:,-1].reshape(-1,1)
# do the fitting 
lr = LinearRegression().fit(np.log(lstat),boston.target)
# plot the results
fig, ax = plt.subplots(figsize=(4,4))
plt.scatter(lstat,boston.target)
# use predict to conveniently plot the line!
plt.scatter(lstat,lr.predict(np.log(lstat)))
plt.xlabel('lstat')
plt.ylabel('housing price')
# equation of the line and explained r^2 in title
plt.title('hp = {:.3f} * log(ls) + {:.3f} | R^2={:.3f}'.format(lr.coef_[0],lr.intercept_,lr.score(np.log(lstat),boston.target)))

Aha - that gave us a comfortable boost in $R^2$! Nice!

Let's try to see what the log did to our data distribution. To measure the skewedness of the distribution mathematically, we are going to use the skewedness measure (who would have thunk it!):

$\gamma = \frac{\frac{1}{n}\sum_{i=1}^{n} (x_i - \bar{x})^3}{\sqrt{\frac{1}{n-1}\sum_{i=1}^{n} (x_i - \bar{x})^3}}$

This is the third standardized moment of a distribution (with the first two moments being "0" and "1" by definition, and the fourth being the kurtosis).

`numpy` does not have skew as a function, so we are going to resort to `pandas`. We could have also used `scipy` for this...

In [ ]:
fig=plt.figure(figsize=(8,4))
fig.add_subplot(121)
# plot histogram
plt.hist(lstat)
# convert to pandas DataFrame, so we have access to skew
tmp=pandas.DataFrame(lstat)
plt.title('LSTAT: skew = {:.3f}'.format(tmp.skew()[0]))
fig.add_subplot(122)
# plot histogram for log data
plt.hist(np.log(lstat))
tmp=pandas.DataFrame(np.log(lstat))
plt.title('log(LSTAT): skew = {:.3f}'.format(tmp.skew()[0]))
plt.show()


So, that seemed to have helped a little with the symmetry. 

But what about the house prices? They may be skewed, too:

In [ ]:
fig=plt.figure(figsize=(8,4))
fig.add_subplot(121)
plt.hist(boston.target)
tmp=pandas.DataFrame(boston.target)
plt.title('Prices: skew = {:.3f}'.format(tmp.skew()[0]))
fig.add_subplot(122)
plt.hist(np.log(boston.target))
tmp=pandas.DataFrame(np.log(boston.target))
plt.title('log(Prices): skew = {:.3f}'.format(tmp.skew()[0]))
plt.show()


They seem to be! So let's do a log-log fit! Let's transform both values logarithmically and then fit a line...

In [ ]:
# one variable from the housing data - since we only have one feature, we need to
# reshape the data, so that LinearRegression properly fits!
lstat = boston.data[:,-1].reshape(-1,1)
# do the log-log fitting 
lr = LinearRegression().fit(np.log(lstat),np.log(boston.target))
# plot the results
fig, ax = plt.subplots(figsize=(4,4))
plt.scatter(np.log(lstat),np.log(boston.target))
# use predict to conveniently plot the line!
plt.scatter(np.log(lstat),lr.predict(np.log(lstat)))
plt.xlabel('lstat')
plt.ylabel('housing price')
# equation of the line and explained r^2 in title
plt.title('log(hp) = {:.3f} * log(ls) + {:.3f} | R^2={:.3f}'.format(lr.coef_[0],lr.intercept_,lr.score(np.log(lstat),np.log(boston.target))))


So that had another (albeit smaller) boost for $R^2$. 

But is this really a robust increase? Maybe fiddling too much with the data will at some point have adverse effects!

We would be able answer this question, if we try to repeat the same things using separate training- and test-sets for the data - see later!

## DIY: $R^2$ competition

Find out which **untransformed** variable has the best explainability in terms of $R^2$-value!

In [ ]:
# go through all variables

    # fit current variable with boston.target

    # put r^2-value in a list
    
# find maximum index of that list and print out winning feature

# Multivariate linear regression

Now, let's try to build a multivariate linear model. We can do this in the exact same way in `scikit`, simply by putting in our multidimensional samples:

In [ ]:
# make full model
lr = LinearRegression().fit(boston.data,boston.target)
# plot the coefficients of the linear fit (13 of them)
fig, ax = plt.subplots(figsize=(4,4))
plt.bar(range(13),lr.coef_)
plt.xticks(range(13),boston.feature_names,rotation='vertical')
plt.title('R^2={:.3f}'.format(lr.score(boston.data,boston.target)))

We observe that the $R^2$-value is much higher now (at 0.74), meaning that the additional variables help in increasing the linear predictability of the house prices.

But is that really surprising? I mean after all instead of giving the algorithm two features (including the intercept) to predict a value, we are now giving it 14! So we have a **lot** more power. In fact, this power may also cause problems, as we are perhaps doing too much. 

## Adjusted $R^2$

People have therefore come up with a way to correct the $R^2$-value depending on the number of feature dimensions:

$R^2_{adj} = R^2 - \frac{p}{n-p-1}(1-R^2)$ with $p$ being number of feature dimensions and $n$ being the number of samples.

In our case, $R^2_{adj} = 0.734$

Also, apparently Nitric Oxides ("NOX") have a **huge** negative effect on housing prices. Or do they? 

## DIY: improve the interpretability of the linear regression

We can do this very easily, namely by...

In [ ]:
# implement the fix to improve interpretability

#
#
fig, ax = plt.subplots(figsize=(4,4))
plt.bar(range(13),lr.coef_)
plt.xticks(range(13),boston.feature_names,rotation='vertical')
plt.title('R^2={:.3f}'.format(lr.score(data,boston.target)))

# Ridge regression

Let's revisit the housing data with ridge regression. The implementation in `scikit` uses the name `alpha` for what we called $\lambda$ as the influence parameter. We will give this a nice range to see what happens.

In [ ]:
from sklearn.linear_model import Ridge

data = boston.data
target = boston.target

# set range of alphas
alphas = np.power(10,np.arange(-5,10,0.25))

# array for holding the r^2 values
r2s = np.zeros_like(alphas)

coefs = []

# go through all alphas
for idx,alpha in enumerate(alphas):
    # do the ridge regression and record r^2
    rr = Ridge(alpha = alpha).fit(data,target)
    r2s[idx]=rr.score(data,target)
    coefs.append(rr.coef_)
    
# plot the r^2-values of the ridge regression fit
fig, ax = plt.subplots(2,1,figsize=(8,4))
plt.subplot(2,1,1)
plt.plot(np.arange(-5,10,0.25),r2s)
plt.subplot(2,1,2)
plt.plot(np.arange(-5,10,0.25),coefs)

Well, as we can see, we can see nothing. The housing data does not benefit whatsoever from doing the ridge regression. The $R^2$-values decrease as we increase `alpha`.

That is weird.

Let's try to get back to the idea of splitting our data into training and test sets. If we only use part of the data for training, and another part for testing, we can use several of such splits to evaluate how good the algorithms can **generalize** from training to testing. 

So, in the following, we train our linear regression and ridge regression models in this way. For each `alpha`-value, we run multiple splits of our big dataset into smaller datasets. We train the regressors on the training set and evaluate their performance on the test set, averaging over the multiple splits in each case.

Let's see how this works:

In [ ]:
from sklearn.linear_model import Ridge

data = boston.data
target = boston.target

# set range of alphas
exps = np.arange(-5,10,0.25)
alphas = np.power(10,exps)

#number of train points
numTrain = 50

# number of reshufflings per alpha value
numShuffle = 20

# array for holding the r^2 values
r2s_train = np.zeros((numShuffle,alphas.size))
r2s_test = np.zeros((numShuffle,alphas.size))
lr2s_train = np.zeros((numShuffle,alphas.size))
lr2s_test = np.zeros((numShuffle,alphas.size))


for it in range(0,numShuffle):
    randIdx = np.arange(0,target.size)
    np.random.shuffle(randIdx)
    trainIdx = randIdx[:numTrain]
    testIdx = randIdx[numTrain:]
    # go through all alphas
    for idx,alpha in enumerate(alphas):
        # do the ridge regression and record r^2
        rr = Ridge(alpha = alpha).fit(data[trainIdx],target[trainIdx])
        r2s_train[it,idx]=rr.score(data[trainIdx],target[trainIdx])
        r2s_test[it,idx]=rr.score(data[testIdx],target[testIdx])
        lr = LinearRegression().fit(data[trainIdx],target[trainIdx])
        lr2s_train[it,idx]=lr.score(data[trainIdx],target[trainIdx])
        lr2s_test[it,idx]=lr.score(data[testIdx],target[testIdx])
    
# plot the r^2-values of the ridge regression fit
fig, ax = plt.subplots(figsize=(8,4))
plt.plot(exps,r2s_train.mean(axis=0),label='rrTrain')
plt.plot(exps,r2s_test.mean(axis=0),label='rrTest')
plt.plot(exps,lr2s_train.mean(axis=0),label='lrTrain')
plt.plot(exps,lr2s_test.mean(axis=0),label='lrTest')
plt.legend(loc='best')
plt.title('found {:d} alpha-values that are better for Ridge in testing'.format(np.size(np.where(r2s_test.mean(axis=0)>lr2s_test.mean(axis=0)))))

There we go!

Now the ridge regression beats out linear regression **on the test set** for certain values of alpha. 

In addition, we see clearly that performance on the test set is **worse** than on the training set. This is the problem of generalization!

By changing the code, we can also see that more training data makes the two algorithms more and more similar. In addition, training and testing performance pull together.